# Project 2: Tracking User Activity

This notebook serves as a step-by-step tuide to build the pipeline, consume messages in Spark, and run transformations to land assessments data so that it could be queried by clients.

### JSON file

**1. access the data by using curl and explore the data structure**

In [32]:
!curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 9096k  100 9096k    0     0  21.0M      0 --:--:-- --:--:-- --:--:-- 58.0M


In [33]:
!cat assessment-attempts-20180128-121051-nested.json | jq '.[0]'

{
  "keen_timestamp": "1516717442.735266",
  "max_attempts": "1.0",
  "started_at": "2018-01-23T14:23:19.082Z",
  "base_exam_id": "37f0a30a-7464-11e6-aa92-a8667f27e5dc",
  "user_exam_id": "6d4089e4-bde5-4a22-b65f-18bce9ab79c8",
  "sequences": {
    "questions": [
      {
        "user_incomplete": true,
        "user_correct": false,
        "options": [
          {
            "checked": true,
            "at": "2018-01-23T14:23:24.670Z",
            "id": "49c574b4-5c82-4ffd-9bd1-c3358faf850d",
            "submitted": 1,
            "correct": true
          },
          {
            "checked": true,
            "at": "2018-01-23T14:23:25.914Z",
            "id": "f2528210-35c3-4320-acf3-9056567ea19f",
            "submitted": 1,
            "correct": true
          },
          {
            "checked": false,
            "correct": true,
            "id": "d1bf026f-554f-4543-bdd2-54dcf105b826"
          }
        ],
        "user_submitted": true,
        "id": "7a2ed6d3-f492-49b

### Kafka: publish and consume messages

**2. spin up cluster using docker-compose (extra step: check the containers with ps)**

In [34]:
!docker-compose up -d

project-2-redcarrott_zookeeper_1 is up-to-date
project-2-redcarrott_cloudera_1 is up-to-date
project-2-redcarrott_mids_1 is up-to-date
project-2-redcarrott_kafka_1 is up-to-date
project-2-redcarrott_spark_1 is up-to-date


In [35]:
!docker-compose ps

         Name                   Command           State           Ports         
--------------------------------------------------------------------------------
project-2-redcarrott_c   cdh_startup_script.sh    Up      11000/tcp, 11443/tcp, 
loudera_1                                                 19888/tcp, 50070/tcp, 
                                                          8020/tcp, 8088/tcp,   
                                                          8888/tcp, 9090/tcp    
project-2-redcarrott_k   /etc/confluent/docker/   Up      29092/tcp, 9092/tcp   
afka_1                   run                                                    
project-2-redcarrott_m   /bin/bash                Up      8888/tcp              
ids_1                                                                           
project-2-redcarrott_s   docker-entrypoint.sh     Up      0.0.0.0:8888->8888/tcp
park_1                   bash                             ,:::8888->8888/tcp    
project-2-redcarrott_z   /et

**3. create assessments topic + check that the topic has been properly created**

In [36]:
!docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

In [37]:
!docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181

Topic: assessments	TopicId: 8hR6gdn8T_ml68OjE4l1uQ	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: assessments	Partition: 0	Leader: 1	Replicas: 1	Isr: 1


**4. publish messages to the assessments topic**

In [50]:
!docker-compose exec mids bash -c "cat /project-2-redcarrot/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments && echo 'Messages published'"

cat: /project-2-redcarrot/assessment-attempts-20180128-121051-nested.json: No such file or directory
Messages published


### Spark

**5. launch Spark session in Jupyter notebook**

In [25]:
!docker-compose exec spark pyspark

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:09:58) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)] on linux
Type "help", "copyright", "credits" or "license" for more information.
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/25 05:42:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/10/25 05:43:09 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 1.2.0
21/10/25 05:43:10 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
21/10/25 05:43:12 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _

In [30]:
!docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root --notebook-dir=/w205/' pyspark

^C
Traceback (most recent call last):
  File "/opt/anaconda3/bin/jupyter-notebook", line 6, in <module>
    sys.exit(notebook.notebookapp.main())
  File "/opt/anaconda3/lib/python3.6/site-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/opt/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "<decorator-gen-7>", line 2, in initialize
  File "/opt/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 87, in catch_config_error
    return method(app, *args, **kwargs)
  File "/opt/anaconda3/lib/python3.6/site-packages/notebook/notebookapp.py", line 1290, in initialize
    super(NotebookApp, self).initialize(argv)
  File "<decorator-gen-6>", line 2, in initialize
  File "/opt/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 87, in catch_config_error
    return met

Note: edited docker-compose.yml to allow access through this port.

**6. transform messages**

6.1 consume messages from kafka into spark

In [31]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe", "assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load()

NameError: name 'spark' is not defined

In [ ]:
raw_assessments.cache()

6.2 look at schema of kafka data import 

In [ ]:
raw_assessments.printSchema()

6.3 